In [ ]:
import pandas as pd
import pymssql
import re
import time

In [ ]:
def reconnect():
    global conn, cursor
    try:
        cursor.close()
        conn.close()
    except:
        pass
    conn = pymssql.connect(
        server='***',
        port=0000,
        user='***',
        password='***',
        database='***'
    )
    cursor = conn.cursor()

In [ ]:
reconnect()

In [ ]:
query = """
SELECT Id, FullName
FROM CustomerDB.dbo.CustomerInfo
WHERE FullName IS NOT NULL
"""
df = pd.read_sql(query, conn)

In [ ]:
invalid_prefixes = ['آقای', 'اقای', 'آقا', 'اقا', 'خانم', 'خانوم', 'سرکار', 'مهندس', 'دکتر']

In [ ]:
def clean_fullname(name):
    if pd.isnull(name):
        return ''

    for prefix in invalid_prefixes:
        name = re.sub(r'(^|\s)' + prefix + r'(\s|$)', ' ', name)

    name = re.sub(r'\d+', '', name)
    name = re.sub(r'[^\w\s\u0600-\u06FF]', '', name)

    words = name.split()
    # Optional filtering disabled:
    # words = [w for w in words if len(w.strip()) >= 2]

    name = ' '.join(words)
    name = re.sub(r'\s+', ' ', name).strip()

    return name

In [ ]:
df['CleanFullName'] = df['FullName'].apply(clean_fullname)

In [ ]:
print(df[['FullName', 'CleanFullName']].head(10))

In [ ]:
update_data = list(zip(df['CleanFullName'], df['Id']))

In [ ]:
update_query = """
UPDATE CustomerDB.dbo.CustomerInfo
SET CleanFullName = %s
WHERE Id = %s
"""

batch_size = 1000
for i in range(0, len(update_data), batch_size):
    batch = update_data[i:i + batch_size]
    try:
        cursor.executemany(update_query, batch)
        conn.commit()
        print(f"{i + len(batch)} records updated...")
    except Exception as e:
        print(f"Error in batch starting from index {i}: {e}")
        reconnect()
        time.sleep(1)

print("All records updated.")
cursor.close()
conn.close()